<a href="https://colab.research.google.com/github/YanaiA/deep-yanai/blob/master/cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.utils import to_categorical
from keras.datasets import cifar10
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.optimizers import Adam
from datetime import datetime
import posixpath

%pip install wandb -q
import wandb
from wandb.keras import WandbCallback


In [ ]:
 #  prepare data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

num_classes = 10

def _fix_inputs(x, y):
    x = x.astype('float32') / 255.0
    y = to_categorical(y, num_classes)
    return x, y

(x_train, y_train) = _fix_inputs(x_train, y_train)
(x_test, y_test) = _fix_inputs(x_test, y_test)
input_shape = x_train.shape[1:]

In [ ]:
# create model
model = Sequential([
    Dense(200, activation='relu', input_shape=input_shape),
    Flatten(),
    Dense(150, activation='relu'),
    Dense(num_classes, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=5e-3), metrics=['accuracy'])

In [ ]:
wandb.login()
wandb.init(project="yanai-test1")

task = 'cifar10'
date = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
checkpoints_path = f'/opt/ml/model/checkpoints/{task}/{date}'
checkpoints_file_format = 'checkpoints.{epoch:02d}-{val_loss:.2f}.hdf5'
model.fit(x_train, y_train, verbose=2,
          batch_size=32, epochs=200, shuffle=True, validation_split=0.2,
          callbacks=[WandbCallback(),
                      ModelCheckpoint(posixpath.join(checkpoints_path, checkpoints_file_format))])
